# 2D gait analysis of top cam data

> Analysis of DLC tracking data of top camera recordings 

In [ ]:
#| default_exp twoD/topcam

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

from typing import Tuple, List, Dict, Optional, Union
from pathlib import Path
import pandas as pd
import numpy as np


from gait_analysis.core import TrackedRecording
from gait_analysis import twoD

In [ ]:
#| export

class Tracked2DRecording(TrackedRecording):
    """
    Very customized subclass of `TrackedRecording` that was designed to run the gait analysis
    on 2D tracking data obtained from a single camera with a top-down-view on the subject.    
    """

    
    def _load_remaining_metadata(self) -> None:
        self.fps = self._get_correct_fps()
        self.framerate = 1/self.fps
        self.metadata = self._retrieve_metadata(filename = filepath.name)
        

    def _get_correct_fps(self) -> int:
        if self.loaded_tracking_df.shape[0] > 25_000:
            fps = 80
        else:
            fps = 30
        return fps


    def _retrieve_metadata(self, filename: str)->Dict:
        """
        Very much dependent on the following file naming convention:
        LLL_FS-SS_YYMMDD_PPP_whatever.h5 (or .csv)
        Where:
            LLL: three digits mouse line code
            FS-SS: the subject ID (must start with capital F) with generation (e.g. F2 for second generation) and the mouse ID as two digits
            YYMMDD: the date of the recording as two digits year (YY), month (MM), and day (DD)
            PPP: the three letter string of the experimental paradigm
        For instance:
            196_F7-27_220826_OTT_whatever.h5
        """
        splits = filename.split('_')
        line_id, mouse_id, date, paradigm_id, cam_id = splits[0], splits[1], splits[2], splits[3][0:3], 'Top'
        self._check_metadata(metadata = (line_id, mouse_id, date, paradigm_id, cam_id))
        metadata = {'recording_date': self.recording_date, 
                    'animal': f'{self.mouse_line}_{self.mouse_id}', 
                    'paradigm': self.paradigm, 
                    'cam': self.cam_id}
        return metadata
    
    
    # ToDo - replace with something more generalizable that could be put to utils
    def _check_metadata(self, 
                        metadata = Tuple[str]
                       ) -> None: 
        animal_line, animal_id, recording_date, paradigm, cam_id = metadata[0], metadata[1], metadata[2], metadata[3], metadata[4]
        self.cam_id = cam_id
        if animal_line not in self.valid_mouse_lines:
            while True:
                entered_input = input(f'Mouse line for {self.filepath}')
                if entered_input in self.valid_mouse_lines:
                    self.mouse_line = entered_input
                    break
                else:
                    print(f'Entered mouse line does not match any of the defined mouse lines. \nPlease enter one of the following lines: {self.valid_mouse_lines}')
        else:
            self.mouse_line = animal_line
        if not animal_id.startswith('F'):
            while True:
                entered_input = input(f'Mouse ID for {self.filepath}')
                if entered_input.startswith('F'):
                    self.mouse_id = entered_input
                    break
                else:
                    print(f'Animal ID has to start with F. Example: F2-14')
        else:
            self.mouse_id = animal_id
        if paradigm not in self.valid_paradigms:
            while True:
                entered_input = input(f'Paradigm for {self.filepath}')
                if entered_input in self.valid_paradigms:
                    self.paradigm = entered_input
                    break
                else:
                    print(f'Entered paradigm does not match any of the defined paradigms. \nPlease enter one of the following paradigms: {self.valid_paradigms}')
        else:
            self.paradigm = paradigm
        try:
            int(recording_date)
            self.recording_date = recording_date
        except:
            while True:
                entered_input = input(f'Recording date for {self.filepath}')
                try:
                    int(recording_date)
                    self.recording_date = recording_date
                    break
                except:
                    print(f'Entered recording date has to be an integer in shape YYMMDD. Example: 220812')


    def preprocess(self,
                   marker_ids_to_compute_coverage: List[str]=['TailBase', 'Snout'], # List of marker_ids on base of which the tracking coverage will be computed
                   coverage_threshold: float=0.75, # If coverage of the above defined markers is lower than this threshold, the recording will be excluded from the analysis and therefore not further processed 
                   max_seconds_to_interpolate: float=0.5, # Maximum time interval in which consecutive nan´s will be interpolated
                   likelihood_threshold: float=0.5, # Minimum prediction likelihood of DLC that is required to acceppt predicted marker position as valid 
                   marker_ids_to_compute_center_of_gravity: List[str]=['TailBase', 'Snout'], # marker ids that will be used to compute the center of gravity
                   relative_maze_normalization_error_tolerance: float=0.25 # relative error that is tolerated when estimating the maze position for normalizing it´s position
                   ) -> None:
        initial_logs_to_add = {'critical_markers_to_compute_coverage': marker_ids_to_compute_coverage,
                               'coverage_threshold': coverage_threshold,
                               'max_seconds_to_interpolate': max_seconds_to_interpolate,
                               'min_likelihood_threshold': likelihood_threshold,
                               'center_of_gravity_based_on': marker_ids_to_compute_center_of_gravity, 
                               'relative_error_tolerance_corner_detection': relative_maze_normalization_error_tolerance}
        window_length = twoD.utils.get_max_odd_n_frames_for_time_interval(fps = self.fps, time_interval = max_seconds_to_interpolate)
        marker_ids_to_preprocess = twoD.utils.get_preprocessing_relevant_marker_ids(df = self.loaded_tracking_df,
                                                                                    marker_ids_to_exclude = self.marker_ids_to_exclude_for_smoothing_and_interpolation)
        smoothed_df = twoD.utils.smooth_tracked_coords_and_likelihood(df = self.loaded_tracking_df, 
                                                                      window_length = window_length,
                                                                      marker_ids = marker_ids_to_preprocess, 
                                                                      polyorder = 3)
        interpolated_df = twoD.utils.interpolate_low_likelihood_intervals(df = smoothed_df, 
                                                                          marker_ids = marker_ids_to_preprocess, 
                                                                          max_interval_length = window_length,
                                                                          framerate = self.framerate)
        interpolated_df_with_cog = twoD.utils.add_new_marker_derived_from_existing_markers(df = interpolated_df,
                                                                                      existing_markers = marker_ids_to_compute_center_of_gravity,
                                                                                      new_marker_id = 'CenterOfGravity',
                                                                                      likelihood_threshold = likelihood_threshold)
        preprocessed_df = twoD.utils.interpolate_low_likelihood_intervals(df = interpolated_df_with_cog, 
                                                                          marker_ids = ['CenterOfGravity'], 
                                                                          max_interval_length = window_length,
                                                                          framerate = self.framerate)        
        coverage_critical_markers = twoD.utils.compute_coverage(df = preprocessed_df,
                                                                critical_marker_ids = marker_ids_to_compute_coverage,
                                                                likelihood_threshold = likelihood_threshold)
        initial_logs_to_add['coverage_critical_markers'] = coverage_critical_markers
        self._add_to_logs(logs_to_add = initial_logs_to_add)
        if coverage_critical_markers >= coverage_threshold:
            normalization_params = self._get_parameters_to_normalize_maze_coordinates(df = preprocessed_df,
                                                                                      relative_error_tolerance = relative_maze_normalization_error_tolerance)
            self.normalized_df = twoD.utils.normalize_df(df = preprocessed_df, normalization_parameters = normalization_params)
            self.bodyparts = self._create_bodypart_objects()
            normalized_maze_corner_coords = self._get_normalized_maze_corners(normalization_parameters = normalization_params)
            coverage_center_of_gravity = twoD.utils.compute_coverage(df = preprocessed_df,
                                                                     critical_marker_ids = ['CenterOfGravity'],
                                                                     likelihood_threshold = likelihood_threshold)
            additional_logs_to_add = {'coverage_CenterOfGravity': coverage_center_of_gravity}
            for key, value in normalization_params.items():
                additional_logs_to_add[key] = value
            for key, value in normalized_maze_corner_coords.items():
                additional_logs_to_add[f'normalized_{key}_coords'] = value
            self._add_to_logs(logs_to_add = additional_logs_to_add)
            
    
    def _add_to_logs(self,
                     logs_to_add: Dict # key-value pairs of things that shall be added to the logs of the recording
                    ) -> None:
        """
        At the end of all the processing, the "logged" data will be saved in the results .xlsx file.
        It thus allows the user to keep track of which settings they used for the respective analysis.
        """
        if hasattr(self, 'logs') == False:
            self.logs = {}
        for key, value in logs_to_add.items():
            assert key not in self.logs.keys(), f'{key} is already in self.logs.keys - adding it would result in overwriting the previous entry; please ensure unique naming'
            self.logs[key] = value 
    
    
    def _get_parameters_to_normalize_maze_coordinates(self, df: pd.DataFrame, relative_error_tolerance: float) -> Dict:
        corners = twoD.utils.get_corner_coords_with_likelihoods(df = df)
        translation_vector = twoD.utils.get_translation_vector(coords_to_become_origin = corners['MazeCornerClosedLeft']['coords'])
        best_result = twoD.utils.evaluate_maze_shape_using_open_corners(corners_and_likelihoods = corners, 
                                                                        tolerance = relative_error_tolerance)
        if best_result['valid']:
            side_id = best_result['side_id']
            logs_to_add = {'maze_normalization_based_on': f'MazeCornerClosed{side_id}_and_MazeCornerOpen{side_id}'}
            conversion_factor = twoD.utils.get_conversion_factor_px_to_cm(coords_point_a = corners[f'MazeCornerClosed{side_id}']['coords'],
                                                                         coords_point_b = corners[f'MazeCornerOpen{side_id}']['coords'],
                                                                         distance_in_cm = 50)
            rotation_angle = twoD.utils.get_rotation_angle_with_open_corner(corners = corners,
                                                                           side_id = best_result['side_id'],
                                                                           translation_vector = translation_vector,
                                                                           conversion_factor = conversion_factor)
        else:
            logs_to_add = {'maze_normalization_based_on': f'MazeCornerClosedRight_and_MazeCornerClosedLeft'}
            conversion_factor = twoD.utils.get_conversion_factor_px_to_cm(coords_point_a = corners['MazeCornerClosedLeft']['coords'],
                                                                         coords_point_b = corners['MazeCornerClosedRight']['coords'],
                                                                         distance_in_cm = 4)
            
            
            rotation_angle = twoD.utils.get_rotation_angle_with_closed_corners_only(corners = corners,
                                                                               translation_vector = translation_vector,
                                                                               conversion_factor = conversion_factor)
        self._add_to_logs(logs_to_add = logs_to_add)
        return {'translation_vector': translation_vector, 'rotation_angle': rotation_angle, 'conversion_factor': conversion_factor}
    
    
    def _create_bodypart_objects(self) -> Dict:
        bodyparts = {}
        for marker_id in twoD.utils.get_all_unique_marker_ids(df = self.normalized_df):
            bodyparts[marker_id] = Bodypart2D(bodypart_id = marker_id, df = self.normalized_df, fps = self.fps)
        return bodyparts
    
    
    def _get_normalized_maze_corners(self, normalization_parameters: Dict) -> Dict:
        normalized_maze_corner_coordinates = {}
        corners = self._get_corner_coords_with_likelihoods(df = self.normalized_df)
        for corner_marker_id in corners.keys():
            normalized_maze_corner_coordinates[corner_marker_id] = corners[corner_marker_id]['coords']
        return normalized_maze_corner_coordinates

In [ ]:
#| export

# ToDo: implement properly. Should probably go to core?
class Bodypart2D:
    """
    Class that contains information for one single Bodypart.
    
    Attributes:
        self.id(str): Deeplabcut label of the bodypart
    """
    
    @property
    def exclusion_criteria(self) -> Dict:
        return {'likelihood_threshold': 0.5,
                              'min_x': -5,
                              'max_x': 55,
                              'min_y': -3,
                              'max_y': 7}

    
    def __init__(self, bodypart_id: str, df: pd.DataFrame, fps: int)->None:
        """ 
        Constructor for class Bodypart. 
        
        Since the points in df_raw represent coordinates in the distorted dataframe, we use df_undistort for calculations.
        
        Parameters:
            bodypart_id(str): unique id of marker
        """
        self.id = bodypart_id
        sliced_df = self._slice_df(df = df)
        self.df = self._apply_exclusion_criteria(df = sliced_df, exclusion_criteria = self.exclusion_criteria)
        self.fps = fps
        self.framerate = 1/fps

        
    def _slice_df(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Function, that extracts the coordinates of a single bodypart.
        
        Parameters:
            df(pandas.DataFrame): the full dataframe of the recording with all bodyparts
        """
        df_input = df.copy()
        data = {'x': df_input.loc[:, f'{self.id}_x'], 
                'y': df_input.loc[:, f'{self.id}_y'], 
                'likelihood': df_input.loc[:, f'{self.id}_likelihood']}
        return pd.DataFrame(data = data)
        
        
    def _apply_exclusion_criteria(self, df: pd.DataFrame, exclusion_criteria: Dict) -> None:
        df.loc[df['likelihood'] < exclusion_criteria['likelihood_threshold'], :] = np.nan
        for coord in ['x', 'y']:
            df.loc[df[coord] < exclusion_criteria[f'min_{coord}'], :] = np.nan
            df.loc[df[coord] > exclusion_criteria[f'max_{coord}'], :] = np.nan
        return df

        
    def calculate_speed_and_identify_immobility(self, sliding_window_size: int, immobility_threshold: float) -> None:
        self._add_speed_to_df()
        self._add_rolling_speed_to_df(sliding_window_size = sliding_window_size)
        self._add_immobility_to_df(immobility_threshold = immobility_threshold)
    
    
    def _add_speed_to_df(self)->None:
        self.df.loc[:, 'speed_cm_per_s'] = (self.df.loc[:, 'x'].diff()**2 + self.df.loc[:, 'y'].diff()**2)**0.5 / self.framerate              
        
    
    def _add_rolling_speed_to_df(self, sliding_window_size: int) -> None:
        min_periods = int(sliding_window_size * 0.66)
        self.df.loc[:, 'rolling_speed_cm_per_s'] = self.df.loc[:, 'speed_cm_per_s'].rolling(sliding_window_size, min_periods = min_periods, center = True).mean()

    
    def _add_immobility_to_df(self, immobility_threshold: float) -> None:
        self.df.loc[:, 'immobility'] = False
        self.df.loc[self.df['rolling_speed_cm_per_s'] < immobility_threshold, 'immobility'] = True

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()